In [ ]:
import requests
import pandas as pd
import os

from dotenv import load_dotenv
api_key = os.getenv('FRED_API_KEY')

base_url = 'https://api.stlouisfed.org/fred/'



In [ ]:

obs_endpoint = 'series/observations'

# Assign parameters
series_id = 'CPIAUCSL'
start_date = '2000-01-01'
end_date = '2023-06-30'
ts_frequency = 'q' # quarterly data
ts_units = 'pc1'

obs_params = {
    'series_id': series_id,
    'api_key': api_key,
    'file_type': 'json',
    'observation_start': start_date,
    'observation_end': end_date,
    # 'frequency': ts_frequency
    # 'units': ts_units
}

In [10]:
response = requests.get(base_url + obs_endpoint, params=obs_params)
print(response.json())

{'realtime_start': '2025-08-19', 'realtime_end': '2025-08-19', 'observation_start': '2000-01-01', 'observation_end': '2023-06-30', 'units': 'lin', 'output_type': 1, 'file_type': 'json', 'order_by': 'observation_date', 'sort_order': 'asc', 'count': 282, 'offset': 0, 'limit': 100000, 'observations': [{'realtime_start': '2025-08-19', 'realtime_end': '2025-08-19', 'date': '2000-01-01', 'value': '169.3'}, {'realtime_start': '2025-08-19', 'realtime_end': '2025-08-19', 'date': '2000-02-01', 'value': '170.0'}, {'realtime_start': '2025-08-19', 'realtime_end': '2025-08-19', 'date': '2000-03-01', 'value': '171.0'}, {'realtime_start': '2025-08-19', 'realtime_end': '2025-08-19', 'date': '2000-04-01', 'value': '170.9'}, {'realtime_start': '2025-08-19', 'realtime_end': '2025-08-19', 'date': '2000-05-01', 'value': '171.2'}, {'realtime_start': '2025-08-19', 'realtime_end': '2025-08-19', 'date': '2000-06-01', 'value': '172.2'}, {'realtime_start': '2025-08-19', 'realtime_end': '2025-08-19', 'date': '2000

In [13]:
if response.status_code == 200:
    res_data = response.json()
    obs_data = pd.DataFrame(res_data['observations'])
    obs_data['date'] = pd.to_datetime(obs_data['date'])
    obs_data.set_index('date', inplace=True)
    obs_data['value'] = obs_data['value'].astype(float)
    print(f"{res_data.keys()}")
else:
    print('Failed to retrieve data. Status code:', response.status_code)


dict_keys(['realtime_start', 'realtime_end', 'observation_start', 'observation_end', 'units', 'output_type', 'file_type', 'order_by', 'sort_order', 'count', 'offset', 'limit', 'observations'])


In [14]:
cat_endpoint = 'category'
cat_id = 0

cat_params = {
    'api_key': api_key,
    'file_type': 'json',
    'category_id': cat_id
}

In [15]:
response = requests.get(base_url + cat_endpoint, params=cat_params)

if response.status_code == 200:
    res_data = response.json()
    
else:
    print('Failed to retrieve data. Status code:', response.status_code)

In [16]:
# Assign endpoint
child_endpoint = 'category/children'
parent_id = 0
# Assign params
child_params = {
    'api_key': api_key,
    'file_type': 'json',
    'category_id': parent_id
}


In [17]:

# Make request to FRED API
response = requests.get(base_url + child_endpoint, params=child_params)

if response.status_code == 200:
    res_data = response.json()
    
else:
    print('Failed to retrieve data. Status code:', response.status_code)


In [18]:

cat_id = '32455'
cat_srs_endpoint = 'category/series'
cat_srs_params = {
    'api_key': api_key,
    'file_type': 'json',
    'category_id': cat_id,
}

response = requests.get(base_url + cat_srs_endpoint, params=cat_srs_params)

if response.status_code == 200:
    cat_srs_data = response.json()
    cat_srs = pd.DataFrame(cat_srs_data['seriess'])
    cat_srs = cat_srs.sort_values(
        by='popularity', 
        ascending=False, 
        ignore_index=True
    )
else:
    print('Failed to retrieve data. Status code:', response.status_code)


In [19]:


rel_id_endpoint = 'releases'
rel_id_params = {
    'api_key': api_key,
    'file_type': 'json',
}

response = requests.get(base_url + rel_id_endpoint, params=rel_id_params)

if response.status_code == 200:
    rel_data = response.json()
    releases = pd.DataFrame(rel_data['releases'])

else:
    print('Failed to retrieve data. Status code:', response.status_code)

# Review data
print(releases[['id', 'name']])


      id                                               name
0      9  Advance Monthly Sales for Retail and Food Serv...
1     10                               Consumer Price Index
2     11                              Employment Cost Index
3     13  G.17 Industrial Production and Capacity Utiliz...
4     14                               G.19 Consumer Credit
..   ...                                                ...
313  736                       Visa Spending Momentum Index
314  737                            National Housing Survey
315  738  Quits and Layoffs to Nonemployment Based on th...
316  739            Kansas City Fed Policy Rate Uncertainty
317  769  Fujita, Moscarini, and Postel-Vinay Employer-t...

[318 rows x 2 columns]


In [26]:
releases.iloc[0].notes

'The U.S. Census Bureau conducts the Advance Monthly Retail Trade and Food Services Survey to provide an early estimate of monthly sales by kind of business for retail and food service firms located in the United States. Each month, questionnaires are mailed to a probability sample of approximately 4,700 employer firms selected from the larger Monthly Retail Trade Survey. Advance sales estimates are computed using a link relative estimator. For each detailed industry, we compute a ratio of current-to previous month weighted sales using data from units for which we have obtained usable responses for both the current and previous month. For each detailed industry, the advance total sales estimates for the current month is computed by multiplying this ratio by the preliminary sales estimate for the previous month (derived from the larger MRTS) at the appropriate industry level. Total estimates for broader industries are computed as the sum of the detailed industry estimates. The link rela

In [20]:

rel_srs_endpoint = 'release/series'
rel_srs_params = {
    'api_key': api_key,
    'file_type': 'json',
    'release_id': 10,
}

response = requests.get(base_url + rel_srs_endpoint, params=rel_srs_params)

if response.status_code == 200:
    rel_srs_data = response.json()
    rel_srs = pd.DataFrame(rel_srs_data['seriess'])

else:
    print('Failed to retrieve data. Status code:', response.status_code)

# Review the data // use limit and offset params to paginate
print(rel_srs)
print(rel_srs[['id','title']])


                 id realtime_start realtime_end  \
0          CPIAPPNS     2025-08-19   2025-08-19   
1          CPIAPPSL     2025-08-19   2025-08-19   
2          CPIAUCNS     2025-08-19   2025-08-19   
3          CPIAUCSL     2025-08-19   2025-08-19   
4          CPIEDUNS     2025-08-19   2025-08-19   
..              ...            ...          ...   
995    CUURA300SAES     2025-08-19   2025-08-19   
996     CUURA300SAF     2025-08-19   2025-08-19   
997    CUURA300SAF1     2025-08-19   2025-08-19   
998   CUURA300SAF11     2025-08-19   2025-08-19   
999  CUURA300SAF116     2025-08-19   2025-08-19   

                                                 title observation_start  \
0    Consumer Price Index for All Urban Consumers: ...        1914-12-01   
1    Consumer Price Index for All Urban Consumers: ...        1947-01-01   
2    Consumer Price Index for All Urban Consumers: ...        1913-01-01   
3    Consumer Price Index for All Urban Consumers: ...        1947-01-01   
4    Co

In [21]:

src_endpoint = 'sources'
src_params = {
    'api_key': api_key,
    'file_type': 'json',
}

response = requests.get(base_url + src_endpoint, params=src_params)

if response.status_code == 200:
    src_data = response.json()
    src = pd.DataFrame(src_data['sources'])
else:
    print('Failed to retrieve data. Status code:', response.status_code)

# Review data // sources are related to releases and vice versa
print(src[['id', 'name']])

      id                                               name
0      1  Board of Governors of the Federal Reserve Syst...
1      3               Federal Reserve Bank of Philadelphia
2      4                  Federal Reserve Bank of St. Louis
3      6  Federal Financial Institutions Examination Cou...
4     11                                Dow Jones & Company
..   ...                                                ...
112  301                                            Ma, Sai
113  367                                               Visa
114  400                                         Fannie Mae
115  434                                 Ellieroth, Kathrin
116  435                                    Michaud, Amanda

[117 rows x 2 columns]
